In [1]:
import pandas as pd
from pathlib import Path
import numpy as np 
%matplotlib inline

In [2]:
# Set file paths
hd_data = Path("./Resources/HD.csv")
jnj_data = Path("./Resources/JNJ.csv")
intc_data = Path("./Resources/INTC.csv")
amd_data = Path("./Resources/AMD.csv")
mu_data = Path("./Resources/MU.csv")
nvda_data = Path("./Resources/NVDA.csv")
tsm_data = Path("./Resources/TSM.csv")


# Read the individual CSV datasets
hd = pd.read_csv(hd_data, index_col="date", infer_datetime_format=True, parse_dates=True)
jnj = pd.read_csv(jnj_data, index_col="date",infer_datetime_format=True, parse_dates=True)
intc = pd.read_csv(intc_data, index_col="date",infer_datetime_format=True, parse_dates=True)
amd = pd.read_csv(amd_data, index_col="date",infer_datetime_format=True, parse_dates=True)
mu = pd.read_csv(mu_data, index_col="date",infer_datetime_format=True, parse_dates=True)
nvda = pd.read_csv(nvda_data, index_col="date",infer_datetime_format=True, parse_dates=True)
tsm = pd.read_csv(tsm_data, index_col="date",infer_datetime_format=True, parse_dates=True)

In [3]:
combined_df = pd.concat([jnj, hd, intc, amd], axis="columns", join="inner")
combined_df.sort_index(inplace=True)
combined_df.head()

combined_df2 = pd.concat([mu,nvda,tsm], axis="columns", join="inner")
combined_df2.sort_index(inplace=True)
combined_df2.head()

# Creates a dictionary of dataframes 

dataframe_dictionary = {"First_df": combined_df,
                        "Second_df" : combined_df2}


 

In [4]:
## Function that calculates log returns  
def calculateLogreturns(df):
    
    # Calculate returns
    returns1 = df/df.shift(1)
    # Creates log returns
    log_returns = np.log(returns1)
    return log_returns

    

In [5]:
## Function that runs Efficient Frontier Simulation and returns weighted index, return, expected return  

def efficient_frontier(calculateLogreturns, number_columns):
        
       ## Sets number of portfolios to be simulated 
        number_portfolios = 10
        
       ## Creates arrays to store calculations
        weight = np.zeros((number_portfolios, number_columns))
        expectedReturn = np.zeros(number_portfolios)
        expectedVolatility = np.zeros(number_portfolios)
        sharpeRatio = np.zeros(number_portfolios)

       ## Creates two variables by calling calculateLogReturns function
        meanLogRet = calculateLogreturns(df).mean()
        sigma = calculateLogreturns(df).cov()

       ## For loop that takes in the number of portfolios  
        for k in range(number_portfolios):
            # Generate random weight vector
            w = np.array(np.random.random(number_columns))
            w = w /np.sum(w)
            weight[k,:] = w
           
        # Expected log return
            expectedReturn[k] = np.sum(meanLogRet * w)
    
            # Expected volatility
            expectedVolatility[k] = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
   
            # Sharpe Ratio
            sharpeRatio[k] = expectedReturn[k]/expectedVolatility[k]
    
        # Maximizes Sharpe ratio 
        WeightIndex = sharpeRatio.argmax()
        
        # Weighted index that maximizes sharpe ratio 
        Final_Index = weight[WeightIndex,:]
        
        return Final_Index, expectedReturn, expectedVolatility

   

In [8]:
# For loop that uses dataframe dictionary to run functions for calculateLogreturns and efficient frontier

for name_of_df, df in dataframe_dictionary.items():
    
    ## Creates variable for column names
    columns = df.columns
    
    ## Defines number of columns in outputdataframe
    shape = df.shape
    number_columns = shape[1]
    
    
    ## Runs calculateLogreturns function
    calculateLogreturns(df)
    
    
    ## Runs efficient frontier function and returns weights, returns, and volatility
    Weights, Returns, Volatility = efficient_frontier(calculateLogreturns, number_columns)

    ## Creates a dataframe of weights from Efficient Frontier simulation
    outputdataframe = pd.DataFrame(Weights)
    outputdataframe = outputdataframe.T
    outputdataframe.columns = columns
    
    
   
   
    
    outputdataframe.to_csv(f"{name_of_df}.csv")



ValueError: Dot product shape mismatch, (2517, 3) vs (4,)

In [7]:
cumulative_returns_df = []


for name_of_df, df in dataframe_dictionary.items():
    daily_returns = df.pct_change()
    cum_returns = daily_returns.dot(Weights)
    cumulative_returns = (1+cum_returns).cumprod()
    cumulative_returns_df.append(cumulative_returns)

for i in cumulative_returns_df:
    i.plot(figsize=(10,5))

ValueError: Dot product shape mismatch, (2517, 4) vs (3,)